In [2]:
import election_metrics as em
import os 
import pandas as pd
import numpy as np

In [3]:

def parse(file):
    data = pd.read_csv(str(file), low_memory=False)

    candidates = []
    
    #number of voters
    n = len(data)

    ballots = {}
    for i in range(1, n):
        ranking = ()
        j = 1
        flag = True
        while flag:
            try:
                c = data.at[i, "rank"+str(j)]
                if c != "skipped" and c != "Write-in"  and c[0:5].lower() != "write" and c != "overvote" and c != "undervote":
                    if c not in ranking:
                        ranking += (c,)
                    if c not in candidates:
                        candidates.append(c)
                j += 1
            except:
                flag = False
    
        if ranking not in ballots:
            ballots[ranking] = 0
        ballots[ranking] += 1
    return ballots, candidates

In [4]:
directory = os.fsencode("dataverse_files")

total = 0
nc = 0
i = 0
d = {}
#df = pd.DataFrame()
rules = ["irv", "plurality", "majority", "condorcet", "approval", "borda", "mini_max", "black", "bucklin"]
criterion = ["Condorcet", "Condorcet_2", "NA", "Chance of Condorcet"]
rules2 = ["irv", "plurality", "majority", "condorcet", "approval", "borda", "total","chance of irv" ]
df_rules = pd.DataFrame(index=rules, columns=criterion)
df_topN = pd.DataFrame(index=rules2, columns=[i for i in range(1, 20)])
for col in df_rules.columns:
    df_rules[col].values[:] = 0
for col in df_topN.columns:
    df_topN[col].values[:] = 0
for file in os.listdir(directory):
    filename = os.fsdecode(file)
    total += 1
    ballots, candidates = parse("C:/Users/mahsh/Documents/GitHub/bugs-in-democracy/team_arrow/cvr_class/dataverse_files/"+filename)
  #  C:\Users\mahsh\Documents\GitHub\bugs-in-democracy\team_arrow\cvr_class\dataverse_files\Alaska_04102020_PRESIDENTOFTHEUNITEDSTATES.csv

    
    e = em.election_metric(ballots, candidates)
    n = len(candidates)
    for r in rules:
        func = getattr(e, r)
        if func() == -1:
            df_rules.loc[[r],["NA"]] += 1
        else:
            if func() == e.condorcet():
                df_rules.loc[[r],["Condorcet"]] +=1 
            if func() == e.condorcet_n(2):
                df_rules.loc[[r],["Condorcet_2"]] += 1
    if total % 50 == 0:
        print(df_rules)
for r in rules:
    df_rules[[r],["Chance of Condorcet"]] = (df_rules[[r]["Condorcet"]]/total)*100

print(df_rules)

        

<>:38: SyntaxWarning: list indices must be integers or slices, not str; perhaps you missed a comma?
<>:38: SyntaxWarning: list indices must be integers or slices, not str; perhaps you missed a comma?


          Condorcet Condorcet_2  NA Chance of Condorcet
irv              49           1   0                   0
plurality        46           4   0                   0
majority         28           0  22                   0
condorcet        50           0   0                   0
approval         49           1   0                   0
borda            42           8   0                   0
mini_max         50           0   0                   0
black            50           0   0                   0
bucklin          46           4   0                   0
          Condorcet Condorcet_2  NA Chance of Condorcet
irv              97           1   0                   0
plurality        93           7   0                   0
majority         51           0  49                   0
condorcet       100           0   0                   0
approval         96           3   0                   0
borda            87          11   0                   0
mini_max        100           0   0             